In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

In [2]:
#read logon.csv into a dataframe
df = pd.read_csv('logon.csv', usecols = ['user', 'date', 'pc', 'activity'])

In [3]:
#functions to parse data

#used to split the date from the time
def split_date(datetime):
    return datetime.date()

def split_time(datetime):
    return datetime.time()

#determine if actions are performed in working hours
def is_working_hours(timestamp):
    time = timestamp.time()
    
    start = datetime.strptime('09:00:00', '%H:%M:%S').time()
    end = datetime.strptime('17:00:00', '%H:%M:%S').time()
    
    # Check if it's a weekday (Monday: 0, Tuesday: 1, ..., Friday: 4)
    if timestamp.weekday() in range(0, 5):
        if (time >= start and time <= end): # Check if it's 9 AM or later
            return "Work Hours"
        return "Off Hours"
    else:
        return 'Weekend'  # It's not a weekday

#if logons/logoffs occur on the same work day/night
def not_on_same_day(working_df):
    notOnSameDay = []
    
    grouped = working_df.groupby('only_date')
    grouped
    
    for date, group in grouped:
        notOnSameDay.append( int( len(group[group.activity == 'Logon']) - len(group[group.activity == 'Logoff']) ) )
    
    Sum = sum(notOnSameDay)
    return Sum 

In [4]:
#convert date column to datetime data type
df['date'] = pd.to_datetime(df['date'], format= '%m/%d/%Y %H:%M:%S')

#split the date and time values
df['only_date'] = df['date'].apply(split_date)
df['only_time'] = df['date'].apply(split_time)

#create a utility boolean column defining if the time is within work hours
df['WorkingHours'] = df.date.apply(is_working_hours)

In [5]:
#group data by user
grouped = df.groupby('user')

In [6]:
#create a dataframe to hold the final parsed values
pd.set_option('display.max_rows', None)
colnames = ['user', 'numlogonDay', 'numlogonNight','numlogoffDay','numlogoffNight', 'numPClogonDay', 'numPClogonNight', 'numPClogoffDay', 'numPClogoffNight', 'onoffNotsameDay', 'onoffNotsameNight']
results = pd.DataFrame(columns = colnames)

#parse the data
for user, group in grouped:
    
    Day = group[group.WorkingHours == 'Work Hours']
    Night = group[group.WorkingHours != 'Work Hours']
      
    #not sure about the PC logon/logoff
    attributes = {
        'user' : user,
        'numlogonDay' : len(Day[Day.activity == 'Logon']),
        'numlogonNight' : len(Night[Night.activity == 'Logon']),
        'numlogoffDay' : len(Day[Day.activity == 'Logoff']),
        'numlogoffNight' : len(Night[Night.activity == 'Logoff']),
        'numPClogonDay' : len(Day[Day.activity == 'Logon'].pc.unique()),
        'numPClogonNight' : len(Night[Night.activity == 'Logon'].pc.unique()),
        'numPClogoffDay' : len(Day[Day.activity == 'Logoff'].pc.unique()),
        'numPClogoffNight' : len(Night[Night.activity == 'Logoff'].pc.unique()),
        'onoffNotsameDay' : not_on_same_day(Day), 
        'onoffNotsameNight' : not_on_same_day(Night)
    }
    #have to go through by date and subtract the logons and logoffs
    #maybe use a separate sorting on the df and 
    results.loc[len(results)] = [attributes[column] for column in colnames]

In [7]:
results

,user,numlogonDay,numlogonNight,numlogoffDay,numlogoffNight,numPClogonDay,numPClogonNight,numPClogoffDay,numPClogoffNight,onoffNotsameDay,onoffNotsameNight
0,AAB0162,0,355,0,356,0,1,0,1,0,-1
1,AAB0398,0,356,356,0,0,1,1,0,-356,356
2,AAC0610,278,387,193,194,1,1,1,1,85,193
3,AAC0668,0,356,190,166,0,1,1,1,-190,190
4,AAC3270,0,356,356,0,0,1,1,0,-356,356
5,AAD2188,0,356,356,0,0,1,1,0,-356,356
6,AAD3030,0,356,356,0,0,1,1,0,-356,356
7,AAF0819,302,363,25,362,25,8,25,7,277,1
8,AAF3937,0,356,193,163,0,1,1,1,-193,193
9,AAG1447,197,159,0,356,1,1,0,1,197,-197


In [8]:
results.to_csv('cleaned_logon_dataset.csv')